In [ ]:
!pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


✅ Step 1: Set up Colab (install libraries)

In [ ]:
!pip install transformers datasets seqeval --quiet


✅ Step 2: Upload your labeled CoNLL file



In [ ]:
from google.colab import files

uploaded = files.upload()


Saving teteye.txt to teteye.txt


✅ Step 3: Manually load your CoNLL data into HuggingFace format
This parser reads the CoNLL format into a list of dictionaries.



In [ ]:
def read_conll_data(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        tokens = []
        tags = []
        for line in file:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens = []
                    tags = []
                continue
            splits = line.split()
            if len(splits) >= 2:
                tokens.append(splits[0])
                tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

tokens, ner_tags = read_conll_data("teteye.txt")


In [ ]:
print(tokens)
print(ner_tags)


[['ኢዴፓ', 'በየክልሉ', 'በሚንቀሳቀስበት', 'ጊዜ', 'ሁሉ', 'የሀገሪቱን', 'አጠቃላይ', 'ሕግእንዲሁም', 'የአካባቢውን', 'ባህልና', 'ቋንቋ', 'አክብሮ', 'በአካባቢው', 'የሚገኙ', 'የፖለቲካ', 'ድርጅቶችንም', 'አክብሮና', 'መብታቸውን', 'ጠብቆ', 'በጨዋነት', 'ያስተምራል', '፣', 'ይማራል"', 'ብለዋል', '።'], ['ይኽው', 'በማር', 'የተለወሰና', 'የእምነቱን', 'ተከታዮች', 'አስተሳሰብ', 'ለማደፍረስ', 'የተዘጋጀ', 'የካሴት', 'ክር', 'በትናንሽ', 'ከተሞችና', 'የገጠር', 'አካባቢዎች', 'በስፋት', 'እንዲሰራች', 'የተደረገው', 'የሃይማኖቱን', 'ምስጢር', 'ጠንቅቀው', 'የማያውቁና', 'እንዲሁ', 'ሃይማኖቱን', 'የተቀበሉ', 'ሰዎችን', 'ለማጥመድ', 'መሆኑ', 'ተጠቁሟል', '።'], ['በባህር', 'ዳር', 'ዩኒቨርስቲና', 'በጅማ', 'መምህራን', 'ኮሌጅ', 'ለነባር', 'ተማሪዎች', 'የምግብና', 'የመኝታ', 'አገልግሎት', 'ሊያቆም', 'እንደሚችል', 'ባለፈው', 'ዓመት', 'የተሰጠው', 'ማሳሰቢያ', 'ከዚህ', 'ዓመት', 'ጀምሮ', 'ተግባራዊ', 'ይሆናል', 'በመባሉ', 'ተማሪዎች', 'ከፍተኛ', 'ስጋትና', 'ጭንቀት', 'ላይ', 'መውደቃቸውን', 'ተጠቆመ', '።'], ['የሻዕቢያው', 'መሪ', 'ለዚህ', 'ማስፈራሪያቸው', 'እንደ', 'አብነት', 'የተጠቀሙበት', 'ኦጋዴንን', 'ሲሆን', 'የኢሕአዴግ', 'መንግሥት', 'ከቅኝ', 'ግዛት', 'ውሎቹ', 'ላፈንግጥ', 'ቢል', 'ያንን', 'ግዛት', 'ለዘለዓለሙ', 'ሊያጣው', 'እንደሚችል', 'ግልጽ', 'ሊሆንለት', 'ይገባል', 'ሲሉ', 'አስጠንቅቀዋል', '።'], ['የዩኒቨርስቲው', 'የተማሪዎች', 'ኅብረት', 'በሙስና', 'መዘፈቁ', 'ተነ

In [ ]:
print(len(tokens))


3465


In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict({
    "tokens": tokens,
    "ner_tags": ner_tags
})

dataset = dataset.train_test_split(test_size=0.2)




✅ Step 5: Create a label list

In [ ]:
label_list = sorted(set(tag for doc in ner_tags for tag in doc))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

# Convert ner_tags to IDs
def encode_labels(example):
    example["labels"] = [label_to_id[tag] for tag in example["ner_tags"]]
    return example

dataset = dataset.map(encode_labels)


Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
label_list = sorted(set(tag for doc in ner_tags for tag in doc))
print(label_list)


['B-LOC', 'B-ORG', 'B-PER', 'B-TIME', 'B-TTL', 'I-LOC', 'I-ORG', 'I-PER', 'I-TIME', 'I-TTL', 'O']


In [ ]:
print(label_list)

['B-LOC', 'B-ORG', 'B-PER', 'B-TIME', 'B-TTL', 'I-LOC', 'I-ORG', 'I-PER', 'I-TIME', 'I-TTL', 'O']


✅ Step 6: Load the tokenizer and model
Use xlm-roberta-base, or you can replace it with Davlan/bert-base-multilingual-cased-ner-hrl or others.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Step 7: Tokenize and align labels

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(example["labels"][word_idx])
        else:
            label_ids.append(example["labels"][word_idx] if label_list[example["labels"][word_idx]].startswith("I-") else example["labels"][word_idx])
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels)


Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
!pip install seqeval


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'seqeval' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'seqeval'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3b0ed622ec6cafffb5d9b10c46911eb8c20dc7c2cb5d322c23d0e5e8f1b52ed5
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


✅ Step 8: Load evaluation metric
python
Copy
Edit


In [ ]:
import evaluate

metric = evaluate.load("seqeval")


✅ Step 9: Metrics function



In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id_to_label[pred] for pred, lab in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[lab] for pred, lab in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


✅ Step 10: Training arguments



In [ ]:
!pip install --upgrade transformers --quiet


In [ ]:
import importlib
import transformers
importlib.reload(transformers)


<module 'transformers' from '/usr/local/lib/python3.11/dist-packages/transformers/__init__.py'>

# New Section

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_amharic_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    logging_dir="./logs"
)


In [ ]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(
        batch["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
    )

    all_labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=i) for i in range(len(batch["tokens"]))):
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(batch["labels"][i][word_idx])
            else:
                current_label = batch["labels"][i][word_idx]
                if label_list[current_label].startswith("I-"):
                    label_ids.append(current_label)
                else:
                    label_ids.append(current_label)

            previous_word_idx = word_idx

        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
sample = tokenized_dataset["train"][0]
print("input_ids:", len(sample["input_ids"]))
print("labels:", len(sample["labels"]))


input_ids: 128
labels: 128


In [ ]:
print(tokenized_dataset["train"])
print(tokenized_dataset["train"])

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2772
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2772
})


✅ Step 11: Trainer setup and training



In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-27-2742730739.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobdeve77 (bobdeve77-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.230800


TrainOutput(global_step=522, training_loss=0.2254139303704331, metrics={'train_runtime': 13929.4611, 'train_samples_per_second': 0.597, 'train_steps_per_second': 0.037, 'total_flos': 543280159521792.0, 'train_loss': 0.2254139303704331, 'epoch': 3.0})

✅ Step 12: Save fine-tuned model

In [ ]:
trainer.evaluate()


{'eval_loss': 0.12261331081390381,
 'eval_precision': 0.6837310195227766,
 'eval_recall': 0.7350746268656716,
 'eval_f1': 0.7084738143403012,
 'eval_accuracy': 0.9603049772066861,
 'eval_runtime': 291.8252,
 'eval_samples_per_second': 2.375,
 'eval_steps_per_second': 0.151,
 'epoch': 3.0}

In [ ]:
from collections import Counter

all_labels = [label for doc in dataset["train"]["labels"] for label in doc]
label_counts = Counter(all_labels)
print({id_to_label[i]: c for i, c in label_counts.items()})


{'O': 1745}


In [ ]:
trainer.save_model("amharic_ner_finetuned")
tokenizer.save_pretrained("amharic_ner_finetuned")


NameError: name 'trainer' is not defined

In [ ]:
!ls


sample_data
